# SQLQuery class

It may arise that you want to use SQL-like queries to interact with your Redis vector database. While Redis does not natively support SQL, the `redisvl` library provides a `SQLQuery` class that allows you to write SQL-like queries that are automatically translated into Redis queries.

The `SQLQuery` class is a wrapper around the `sql-redis` package, which provides a SQL-to-Redis query translator. The `sql-redis` package is not installed by default with `redisvl`, so you will need to install with the optional syntax:

In [22]:
%pip install redisvl[sql]

zsh:1: no matches found: redisvl[sql]
Note: you may need to restart the kernel to use updated packages.


## Create an index to search

In [ ]:
from redisvl.utils.vectorize import HFTextVectorizer

hf = HFTextVectorizer()

schema = {
    "index": {
        "name": "user_simple",
        "prefix": "user_simple_docs",
    },
    "fields": [
        {"name": "user", "type": "tag"},
        {"name": "credit_score", "type": "tag"},
        {"name": "job", "type": "tag"},
        {"name": "job_description", "type": "text"},
        {"name": "age", "type": "numeric"},
    ]
}

        {
            "name": "job_embedding",
            "type": "vector",
            "attrs": {
                "dims": len(hf.embed("get embed length")),
                "distance_metric": "cosine",
                "algorithm": "flat",
                "datatype": "float32"
            }
        }

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 1515.30it/s, Materializing param=pooler.dense.weight]                        
MPNetModel LOAD REPORT from: sentence-transformers/all-mpnet-base-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


## Sample Dataset Preparation

Below, create a mock dataset with `user`, `job`, `age`, `credit_score`, and
`user_embedding` fields. The `user_embedding` vectors are synthetic examples
for demonstration purposes.

For more information on creating real-world embeddings, refer to this
[article](https://mlops.community/vector-similarity-search-from-basics-to-production/).

In [ ]:
data = [
    {
        'user': 'john',
        'age': 34,
        'job': 'software engineer',
        'credit_score': 'high',
        'job_description': 'Designs, develops, and maintains software applications and systems.'
    },
    {
        'user': 'bill',
        'age': 54,
        'job': 'engineer',
        'credit_score': 'low',
        'job_description': 'Applies scientific and mathematical principles to solve technical problems.'
    },
    {
        'user': 'mary',
        'age': 24,
        'job': 'doctor',
        'credit_score': 'low',
        'job_description': 'Diagnoses and treats illnesses, injuries, and other medical conditions in the healthcare field.'
    },
    {
        'user': 'joe',
        'age': 17,
        'job': 'dentist',
        'credit_score': 'medium',
        'job_description': 'Provides oral healthcare including diagnosing and treating teeth and gum issues.'
    },
    {
        'user': 'stacy',
        'age': 61,
        'job': 'project manager',
        'credit_score': 'high',
        'job_description': 'Plans, organizes, and oversees projects from inception to completion.'
    }
]

data = [
    {  
        **d,
        "user_embedding": hf.embed(f"{d['job_description']=} {d['job']=}", as_buffer=True),
    } 
    for d in data
]

## Create a `SearchIndex`

With the schema and sample dataset ready, create a `SearchIndex`.

### Bring your own Redis connection instance

This is ideal in scenarios where you have custom settings on the connection instance or if your application will share a connection pool:

In [9]:
from redisvl.index import SearchIndex
from redis import Redis

client = Redis.from_url("redis://localhost:6379")
index = SearchIndex.from_dict(schema, redis_client=client, validate_on_load=True)

### Let the index manage the connection instance

This is ideal for simple cases:

In [10]:
index = SearchIndex.from_dict(schema, redis_url="redis://localhost:6379", validate_on_load=True)

### Create the index

Now that we are connected to Redis, we need to run the create command.

In [11]:
index.create(overwrite=True, drop=True)

## Load Data to `SearchIndex`

Load the sample dataset to Redis.

### Validate data entries on load
RedisVL uses pydantic validation under the hood to ensure loaded data is valid and confirms to your schema. This setting is optional and can be configured in the `SearchIndex` class.

In [12]:
keys = index.load(data)

print(keys)

['user_simple_docs:01KG8A5H6C8KQEZK0H56NEXEVG', 'user_simple_docs:01KG8A5H6GCPS1HD6S2FKWMBRS', 'user_simple_docs:01KG8A5H6H51C0X27K7RP5YKV5', 'user_simple_docs:01KG8A5H6H51C0X27K7RP5YKV6', 'user_simple_docs:01KG8A5H6H51C0X27K7RP5YKV7']


## Create a `SQLQuery` Object

First, let's test a simple select statement such as the one below.

In [13]:
from redisvl.query import SQLQuery

sql_str = """
    SELECT user, credit_score, job, age
    FROM user_simple
    WHERE age > 17
    """

sql_query = SQLQuery(sql_str) 

## Check the created query string

In [14]:
sql_query.redis_query_string(redis_url="redis://localhost:6379")

'FT.SEARCH user_simple "@age:[(17 +inf]" RETURN 4 user credit_score job age'

### Executing the query

In [15]:
results = index.query(sql_query)
results

[{'user': 'john',
  'credit_score': 'high',
  'job': 'software engineer',
  'age': '34'},
 {'user': 'bill', 'credit_score': 'low', 'job': 'engineer', 'age': '54'},
 {'user': 'mary', 'credit_score': 'low', 'job': 'doctor', 'age': '24'},
 {'user': 'stacy',
  'credit_score': 'high',
  'job': 'project manager',
  'age': '61'}]

## Additional query support

### Conditional operators

In [16]:
sql_str = """
    SELECT user, credit_score, job, age
    FROM user_simple
    WHERE age > 17 and credit_score = 'high'
"""

# could maybe be nice to set a connection string at the class level
# this would deviate from our other query like classes though so thinking on it
sql_query = SQLQuery(sql_str)
redis_query = sql_query.redis_query_string(redis_url="redis://localhost:6379")
print("Resulting redis query: ", redis_query)
results = index.query(sql_query)
results

Resulting redis query:  FT.SEARCH user_simple "@age:[(17 +inf] @credit_score:{high}" RETURN 4 user credit_score job age


[{'user': 'john',
  'credit_score': 'high',
  'job': 'software engineer',
  'age': '34'},
 {'user': 'stacy',
  'credit_score': 'high',
  'job': 'project manager',
  'age': '61'}]

In [33]:
sql_str = """
    SELECT user, credit_score, job, age
    FROM user_simple
    WHERE credit_score = 'high' or credit_score = 'low'
    """

sql_query = SQLQuery(sql_str)
redis_query = sql_query.redis_query_string(redis_url="redis://localhost:6379")
print("Resulting redis query: ", redis_query)
results = index.query(sql_query)
results

Resulting redis query:  FT.SEARCH user_simple "((@credit_score:{high})|(@credit_score:{low}))" RETURN 4 user credit_score job age


[{'user': 'john', 'credit_score': 'high', 'job': 'engineer', 'age': '34'},
 {'user': 'stacy', 'credit_score': 'high', 'job': 'dentist', 'age': '61'},
 {'user': 'mary', 'credit_score': 'low', 'job': 'doctor', 'age': '24'},
 {'user': 'bill', 'credit_score': 'low', 'job': 'engineer', 'age': '54'},
 {'user': 'bill', 'credit_score': 'low', 'job': 'engineer', 'age': '54'},
 {'user': 'stacy', 'credit_score': 'high', 'job': 'dentist', 'age': '61'},
 {'user': 'stacy', 'credit_score': 'high', 'job': 'dentist', 'age': '61'},
 {'user': 'john', 'credit_score': 'high', 'job': 'engineer', 'age': '34'},
 {'user': 'mary', 'credit_score': 'low', 'job': 'doctor', 'age': '24'},
 {'user': 'mary', 'credit_score': 'low', 'job': 'doctor', 'age': '24'}]

In [ ]:
# job is a tag field therefore this syntax works
sql_str = """
    SELECT user, credit_score, job, age
    FROM user_simple
    WHERE job IN ('software engineer', 'engineer', 'pancake tester')
    """

# could maybe be nice to set a connection string at the class level
# this would deviate from our other query like classes though so thinking on it
sql_query = SQLQuery(sql_str)
redis_query = sql_query.redis_query_string(redis_url="redis://localhost:6379")
print("Resulting redis query: ", redis_query)
results = index.query(sql_query)
results

Resulting redis query:  FT.SEARCH user_simple "@job:{software engineer|engineer|pancake tester}" RETURN 4 user credit_score job age


[{'user': 'john',
  'credit_score': 'high',
  'job': 'software engineer',
  'age': '34'},
 {'user': 'bill', 'credit_score': 'low', 'job': 'engineer', 'age': '54'}]

### Text based searches

See [the docs](https://redis.io/docs/latest/develop/ai/search-and-query/query/full-text/) for available text queries in Redis.

For more on exact matching see [here](https://redis.io/docs/latest/develop/ai/search-and-query/query/exact-match/)

In [22]:
# Prefix
sql_str = """
    SELECT user, credit_score, job, job_description, age
    FROM user_simple
    WHERE job_description = 'sci*'
"""

# could maybe be nice to set a connection string at the class level
# this would deviate from our other query like classes though so thinking on it
sql_query = SQLQuery(sql_str)
redis_query = sql_query.redis_query_string(redis_url="redis://localhost:6379")
print("Resulting redis query: ", redis_query)
results = index.query(sql_query)
results

Resulting redis query:  FT.SEARCH user_simple "@job_description:sci*" RETURN 5 user credit_score job job_description age


[{'user': 'bill',
  'credit_score': 'low',
  'job': 'engineer',
  'job_description': 'Applies scientific and mathematical principles to solve technical problems.',
  'age': '54'}]

In [23]:
# Suffix
sql_str = """
    SELECT user, credit_score, job, job_description, age
    FROM user_simple
    WHERE job_description = '*care'
"""

# could maybe be nice to set a connection string at the class level
# this would deviate from our other query like classes though so thinking on it
sql_query = SQLQuery(sql_str)
redis_query = sql_query.redis_query_string(redis_url="redis://localhost:6379")
print("Resulting redis query: ", redis_query)
results = index.query(sql_query)
results

Resulting redis query:  FT.SEARCH user_simple "@job_description:*care" RETURN 5 user credit_score job job_description age


[{'user': 'joe',
  'credit_score': 'medium',
  'job': 'dentist',
  'job_description': 'Provides oral healthcare including diagnosing and treating teeth and gum issues.',
  'age': '17'}]

In [26]:
# Fuzzy
sql_str = """
    SELECT user, credit_score, job, job_description, age
    FROM user_simple
    WHERE job_description = '%diagnose%'
"""

# could maybe be nice to set a connection string at the class level
# this would deviate from our other query like classes though so thinking on it
sql_query = SQLQuery(sql_str)
redis_query = sql_query.redis_query_string(redis_url="redis://localhost:6379")
print("Resulting redis query: ", redis_query)
results = index.query(sql_query)
results

Resulting redis query:  FT.SEARCH user_simple "@job_description:%diagnose%" RETURN 5 user credit_score job job_description age


[{'user': 'mary',
  'credit_score': 'low',
  'job': 'doctor',
  'job_description': 'Diagnoses and treats illnesses, injuries, and other medical conditions.',
  'age': '24'}]

In [ ]:
# Phrase no stop words
sql_str = """
    SELECT user, credit_score, job, job_description, age
    FROM user_simple
    WHERE job_description = 'healthcare including'
"""

# could maybe be nice to set a connection string at the class level
# this would deviate from our other query like classes though so thinking on it
sql_query = SQLQuery(sql_str)
redis_query = sql_query.redis_query_string(redis_url="redis://localhost:6379")
print("Resulting redis query: ", redis_query)
results = index.query(sql_query)
results

Resulting redis query:  FT.SEARCH user_simple "@job_description:"healthcare including"" RETURN 5 user credit_score job job_description age


[{'user': 'joe',
  'credit_score': 'medium',
  'job': 'dentist',
  'job_description': 'Provides oral healthcare including diagnosing and treating teeth and gum issues.',
  'age': '17'}]

In [ ]:
# Phrase with stop words currently limitation of core Redis
sql_str = """
    SELECT user, credit_score, job, job_description, age
    FROM user_simple
    WHERE job_description = 'diagnosing and treating'
"""

# could maybe be nice to set a connection string at the class level
# this would deviate from our other query like classes though so thinking on it
sql_query = SQLQuery(sql_str)
redis_query = sql_query.redis_query_string(redis_url="redis://localhost:6379")
print("Resulting redis query: ", redis_query)
results = index.query(sql_query)
results

Resulting redis query:  FT.SEARCH user_simple "@job_description:"diagnosing $and treating"" RETURN 5 user credit_score job job_description age


ResponseError: Syntax error at offset 29 near and

In [35]:
sql_str = """
    SELECT user, credit_score, job, age
    FROM user_simple
    WHERE age BETWEEN 40 and 60
    """

# could maybe be nice to set a connection string at the class level
# this would deviate from our other query like classes though so thinking on it
sql_query = SQLQuery(sql_str)
redis_query = sql_query.redis_query_string(redis_url="redis://localhost:6379")
print("Resulting redis query: ", redis_query)
results = index.query(sql_query)
results

Resulting redis query:  FT.SEARCH user_simple "@age:[40 60]" RETURN 4 user credit_score job age


[{'user': 'bill', 'credit_score': 'low', 'job': 'engineer', 'age': '54'},
 {'user': 'bill', 'credit_score': 'low', 'job': 'engineer', 'age': '54'},
 {'user': 'bill', 'credit_score': 'low', 'job': 'engineer', 'age': '54'},
 {'user': 'bill', 'credit_score': 'low', 'job': 'engineer', 'age': '54'},
 {'user': 'bill', 'credit_score': 'low', 'job': 'engineer', 'age': '54'}]

### Aggregations

See docs for redis supported reducer functions: [https://redis.io/docs/latest/develop/ai/search-and-query/advanced-concepts/aggregations/#supported-groupby-reducers](docs).

In [44]:
sql_str = """
    SELECT
        user,
        COUNT(age) as count_age,
        COUNT_DISTINCT(age) as count_distinct_age,
        MIN(age) as min_age,
        MAX(age) as max_age,
        AVG(age) as avg_age,
        STDEV(age) as std_age,
        FIRST_VALUE(age) as fist_value_age,
        ARRAY_AGG(age) as to_list_age,
        QUANTILE(age, 0.99) as quantile_age
    FROM user_simple
    GROUP BY credit_score
    """

sql_query = SQLQuery(sql_str)
redis_query = sql_query.redis_query_string(redis_url="redis://localhost:6379")
print("Resulting redis query: ", redis_query)
results = index.query(sql_query)
results

Resulting redis query:  FT.AGGREGATE user_simple "*" LOAD 2 age credit_score GROUPBY 1 @credit_score REDUCE COUNT 0 AS count_age REDUCE COUNT_DISTINCT 1 @age AS count_distinct_age REDUCE MIN 1 @age AS min_age REDUCE MAX 1 @age AS max_age REDUCE AVG 1 @age AS avg_age REDUCE STDDEV 1 @age AS std_age REDUCE FIRST_VALUE 1 @age AS fist_value_age REDUCE TOLIST 1 @age AS to_list_age REDUCE QUANTILE 2 @age 0.99 AS quantile_age


[{'credit_score': 'high',
  'count_age': '10',
  'count_distinct_age': '2',
  'min_age': '34',
  'max_age': '61',
  'avg_age': '47.5',
  'std_age': '14.2302494708',
  'fist_value_age': '34',
  'to_list_age': [b'34', b'61'],
  'quantile_age': '61'},
 {'credit_score': 'medium',
  'count_age': '5',
  'count_distinct_age': '1',
  'min_age': '17',
  'max_age': '17',
  'avg_age': '17',
  'std_age': '0',
  'fist_value_age': '17',
  'to_list_age': [b'17'],
  'quantile_age': '17'},
 {'credit_score': 'low',
  'count_age': '10',
  'count_distinct_age': '2',
  'min_age': '24',
  'max_age': '54',
  'avg_age': '39',
  'std_age': '15.8113883008',
  'fist_value_age': '24',
  'to_list_age': [b'24', b'54'],
  'quantile_age': '54'}]

### Vector search

In [37]:
sql_str = """
    SELECT user, vector_distance(user_embedding, :vec) AS vector_distance
    FROM user_simple
    ORDER BY vector_distance ASC
    """
vec = np.array([1, 1, 1], dtype=np.float32).tobytes()
sql_query = SQLQuery(sql_str, params={"vec": vec})

redis_query = sql_query.redis_query_string(redis_url="redis://localhost:6379")
print("Resulting redis query: ", redis_query)
results = index.query(sql_query)

results

Resulting redis query:  FT.SEARCH user_simple "*=>[KNN 10 @user_embedding $vector AS vector_distance]" PARAMS 2 vector $vector DIALECT 2 RETURN 2 user vector_distance SORTBY vector_distance ASC


[{'vector_distance': '0.10912925005', 'user': 'john'},
 {'vector_distance': '0.10912925005', 'user': 'john'},
 {'vector_distance': '0.10912925005', 'user': 'john'},
 {'vector_distance': '0.10912925005', 'user': 'john'},
 {'vector_distance': '0.10912925005', 'user': 'john'},
 {'vector_distance': '0.121690034866', 'user': 'bill'},
 {'vector_distance': '0.121690034866', 'user': 'bill'},
 {'vector_distance': '0.121690034866', 'user': 'bill'},
 {'vector_distance': '0.121690034866', 'user': 'bill'},
 {'vector_distance': '0.121690034866', 'user': 'bill'}]

In [ ]:
sql_str = """
    SELECT user, credit_score, vector_distance(user_embedding, :vec) AS vector_distance
    FROM user_simple
    WHERE credit_score = 'low'
    ORDER BY vector_distance ASC
    """
vec = np.array([1, 1, 1], dtype=np.float32).tobytes()
sql_query = SQLQuery(sql_str, params={"vec": vec})

# they want full string
# i.e. "bank of america" 
# "bank" | "of" | "america"
redis_query = sql_query.redis_query_string(redis_url="redis://localhost:6379")
print("Resulting redis query: ", redis_query)
results = index.query(sql_query)

results

Resulting redis query:  FT.SEARCH user_simple "(@credit_score:{low})=>[KNN 10 @user_embedding $vector AS vector_distance]" PARAMS 2 vector $vector DIALECT 2 RETURN 3 user credit_score vector_distance SORTBY vector_distance ASC


[{'vector_distance': '0.121690034866', 'user': 'bill', 'credit_score': 'low'},
 {'vector_distance': '0.121690034866', 'user': 'bill', 'credit_score': 'low'},
 {'vector_distance': '0.121690034866', 'user': 'bill', 'credit_score': 'low'},
 {'vector_distance': '0.121690034866', 'user': 'bill', 'credit_score': 'low'},
 {'vector_distance': '0.121690034866', 'user': 'bill', 'credit_score': 'low'},
 {'vector_distance': '0.222222208977', 'user': 'mary', 'credit_score': 'low'},
 {'vector_distance': '0.222222208977', 'user': 'mary', 'credit_score': 'low'},
 {'vector_distance': '0.222222208977', 'user': 'mary', 'credit_score': 'low'},
 {'vector_distance': '0.222222208977', 'user': 'mary', 'credit_score': 'low'},
 {'vector_distance': '0.222222208977', 'user': 'mary', 'credit_score': 'low'}]

In [ ]:
sql_str = """
    SELECT user, cosine_distance(user_embedding, :vec) AS vector_distance
    FROM user_simple
    ORDER BY vector_distance DESC
    """

vec = np.array([0.5, 0.1, 0.5], dtype=np.float32).tobytes()
sql_query = SQLQuery(sql_str, params={"vec": vec})

redis_query = sql_query.redis_query_string(redis_url="redis://localhost:6379")
print("Resulting redis query: ", redis_query)
results = index.query(sql_query)

results

Resulting redis query:  FT.SEARCH user_simple "*=>[KNN 10 @user_embedding $vector AS vector_distance]" PARAMS 2 vector $vector DIALECT 2 RETURN 2 user vector_distance SORTBY vector_distance DESC


[{'vector_distance': '0.0295688509941', 'user': 'bill'},
 {'vector_distance': '0.0295688509941', 'user': 'bill'},
 {'vector_distance': '0.0295688509941', 'user': 'bill'},
 {'vector_distance': '0.0295688509941', 'user': 'bill'},
 {'vector_distance': '0.0295688509941', 'user': 'bill'},
 {'vector_distance': '0.00608772039413', 'user': 'john'},
 {'vector_distance': '0.00608772039413', 'user': 'john'},
 {'vector_distance': '0.00608772039413', 'user': 'john'},
 {'vector_distance': '0.00608772039413', 'user': 'john'},
 {'vector_distance': '0.00608772039413', 'user': 'john'}]

## Cleanup

Below we will clean up after our work. First, you can flush all data from Redis associated with the index by
using the `.clear()` method. This will leave the secondary index in place for future insertions or updates.

But if you want to clean up everything, including the index, just use `.delete()`
which will by default remove the index AND the underlying data.

In [39]:
# Clear all data from Redis associated with the index
# await index.clear()

In [40]:
# Butm the index is still in place
# await index.exists()

In [41]:
# Remove / delete the index in its entirety
# await index.delete()